In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model

2025-05-26 14:37:51.135981: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748270271.222820    5605 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748270271.249613    5605 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-26 14:37:51.427219: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Check TensorFlow version and GPU availability
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.18.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# model2=load_model("bpm_model_v2.keras")

In [4]:
import numpy as np
import pandas as pd
!pip install joblib
import joblib 

/bin/bash: /home/kalicharan_ubuntu/miniconda3/envs/tf_gpu/lib/libtinfo.so.6: no version information available (required by /bin/bash)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
X1=np.load("input1_reshaped.npy")
Y1=np.load("out1_reshaped.npy")

X2=np.load("input2_reshaped.npy")
Y2=np.load("out2_reshaped.npy")

In [6]:
X1

array([[[117.54315336],
        [117.54315336],
        [117.2005136 ],
        ...,
        [117.28275571],
        [117.28275571],
        [117.41480498]],

       [[117.41480498],
        [117.41480498],
        [117.29173759],
        ...,
        [118.23561728],
        [118.23561728],
        [118.31317935]],

       [[118.31317935],
        [118.37470463],
        [118.37470463],
        ...,
        [119.79420653],
        [120.12570888],
        [120.12570888]],

       ...,

       [[100.69482022],
        [100.69482022],
        [101.64573269],
        ...,
        [ 99.62628667],
        [ 99.62628667],
        [ 98.59385419]],

       [[ 98.59385419],
        [ 98.42204435],
        [ 98.42204435],
        ...,
        [ 97.53497031],
        [ 98.18329085],
        [ 98.18329085]],

       [[ 98.18329085],
        [ 97.99323596],
        [ 97.99323596],
        ...,
        [ 97.91814895],
        [ 97.94346222],
        [ 97.94346222]]])

In [7]:
Y1

array([[ 75],
       [ 75],
       [ 75],
       ...,
       [100],
       [ 99],
       [ 99]])

In [8]:
X2

array([[[119.88282422],
        [118.97290123],
        [118.97290123],
        ...,
        [120.44384736],
        [120.20303872],
        [119.99503287]],

       [[119.99503287],
        [119.87658634],
        [119.87658634],
        ...,
        [120.22124178],
        [120.01143659],
        [120.01143659]],

       [[119.47442282],
        [119.17043911],
        [119.24923608],
        ...,
        [118.64247921],
        [119.06725013],
        [119.06725013]],

       ...,

       [[132.44560167],
        [132.29570582],
        [132.31129052],
        ...,
        [132.2343775 ],
        [132.2528441 ],
        [132.50644531]],

       [[132.49823978],
        [132.17023003],
        [132.54125   ],
        ...,
        [132.14235988],
        [132.44257552],
        [132.17748507]],

       [[132.350625  ],
        [132.33976563],
        [132.43261719],
        ...,
        [132.08227523],
        [132.08227523],
        [132.37498553]]])

In [9]:
np.unique(Y2)

array([ 97,  99, 102, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118])

In [10]:
np.unique(Y1)

array([ 62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,
        75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
        88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
       101, 102, 103, 104, 105, 106, 107, 108, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
       128, 130, 131, 133, 134, 135, 136, 137, 140, 143, 144, 145, 147,
       148])

In [11]:
X=np.vstack([X1,X2])
Y=np.vstack([Y1,Y2])

In [12]:
X.shape

(5859, 18, 1)

In [13]:
callback=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

In [14]:
# model2.compile(optimizer="adam",metrics=["mae"],loss="mae")
# model2.fit(X,Y,epochs=200,callbacks=callback,validation_split=0.3)

In [15]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dropout, BatchNormalization

def build_cnn_lstm_model():
    inputs = tf.keras.Input(shape=(18, 1))  # Shape: (timesteps, features)

    # CNN part
    x = layers.Conv1D(32, kernel_size=3, padding="same", activation="relu")(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)

    x = layers.Conv1D(64, kernel_size=3, padding="same", activation="relu")(x)
    x = layers.MaxPooling1D(pool_size=2)(x)

    # LSTM part
    x = layers.LSTM(64, return_sequences=True, activation="relu")(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)

    x = layers.LSTM(128, return_sequences=True, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)

    x = layers.LSTM(128, return_sequences=False, activation="relu")(x)  # Final LSTM layer
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)

    # Dense layers
    x = layers.Dense(256, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)

    x = layers.Dense(256, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)

    x = layers.Dense(512, activation="relu")(x)
    outputs = layers.Dense(1)(x)  # Regression output (e.g., BPM)

    model = tf.keras.Model(inputs, outputs)
    return model


In [16]:
# model3=build_cnn_lstm_model()

In [17]:
# model3.compile(optimizer="adam",metrics=["mae"],loss="mae")
# model3.fit(X,Y,epochs=200,callbacks=callback)

In [21]:
model4=build_cnn_lstm_model()
model4.compile(optimizer="adam",metrics=["mae"],loss="mae")
model4.fit(X,Y,epochs=100,callbacks=callback)

Epoch 1/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 29s 59ms/step - loss: 69.3410 - mae: 69.3410 
Epoch 2/100
  5/184 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 13.3948 - mae: 13.3948 

/home/kalicharan_ubuntu/miniconda3/envs/tf_gpu/lib/python3.9/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae
  current = self.get_monitor_value(logs)


184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 13.4488 - mae: 13.4488
Epoch 3/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 13.2507 - mae: 13.2507
Epoch 4/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 12.6729 - mae: 12.6729
Epoch 5/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 12.5608 - mae: 12.5608
Epoch 6/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 11.9143 - mae: 11.9143
Epoch 7/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 12.5025 - mae: 12.5025
Epoch 8/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 12.1456 - mae: 12.1456
Epoch 9/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 12.2180 - mae: 12.2180
Epoch 10/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 12.1481 - mae: 12.1481
Epoch 11/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 12.3161 - mae: 12.3161
Epoch 12/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 11.9883 - mae: 11.9883
Epoch 13/100
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 12.0163 - mae

In [22]:
# model3.save("bpm_model_hm.keras")
model4.save("bpm_model_reshaped.keras")

In [23]:
X.shape

(5859, 18, 1)